In [2]:
from pymongo import MongoClient
from pprint import pprint

In [3]:
client = MongoClient('mongodb://localhost:27017/')
db = client.gym
collection = db["Sportifs"]

In [ ]:
db.list_collection_names()

In [ ]:
pprint(collection.find_one({"Nom":"KERVADEC"}))

In [ ]:
for post in collection.find():
    pprint(post)

In [ ]:
for i in collection.find({"$or":[{"Age":32},{"Age":40}]}):
    pprint(i)

In [ ]:
for i in db.Sportifs.find({ "Sports.Jouer" : "Basket ball" },{"_id":0, "Nom": 1}):
    pprint(i)

In [ ]:
db.Sportifs.count({"Sexe":"F"})

In [ ]:
db.Gymnases.find_one()

In [ ]:
db.Sportifs.find_one()

Trouver la surface moyenne des salles gymnases:

In [133]:
for i in db.Gymnases.aggregate([{"$group": {"_id":"null","moyenne_surface": {"$avg":"$Surface"}}}]):
    pprint (i)

{'_id': 'null', 'moyenne_surface': 444.2857142857143}


Trouver le le nombre de gymnases par ville, afficher le résultat trié
par ordre décroissant:

In [ ]:
varGroup={"$group":{"_id":"$Ville","nb_Gymnase":{"$sum":1}}}
varSort = { "$sort" : { "IdGymnase" : -1 } }
for i in db.Gymnases.aggregate([varGroup,varSort]):
    print(i)

Calculer le nombre de séances journalières:

In [60]:
for i in db.Gymnases.aggregate([{ "$unwind" : "$Seances" },{ "$group": {
        "_id": {
            "Jour": { "$toLower": "$Seances.Jour"} 
        },
        "nb": { "$sum": 1 }
    }}
    ]):
    pprint(i)

{'_id': {'Jour': 'jeudi'}, 'nb': 6}
{'_id': {'Jour': 'mercredi'}, 'nb': 13}
{'_id': {'Jour': 'dimanche'}, 'nb': 17}
{'_id': {'Jour': 'lundi'}, 'nb': 23}
{'_id': {'Jour': 'vendredi'}, 'nb': 6}
{'_id': {'Jour': 'samedi'}, 'nb': 9}
{'_id': {'Jour': 'mardi'}, 'nb': 8}


Quels sont les sportifs (identifiant, nom et prénom) qui ont entre 20 et 30 ans ?

In [61]:
for i in db.Sportifs.aggregate([
    { "$match": { "Age": { "$gte": 20, "$lt": 30 } } },
    { "$project": {"_id":0,"IdSportif":1, "Nom": 1, "Prenom": 1} } ]):
    pprint(i)

{'IdSportif': 3, 'Nom': 'HUE', 'Prenom': 'Pascale'}
{'IdSportif': 5, 'Nom': 'COMES', 'Prenom': 'Sylvie'}
{'IdSportif': 7, 'Nom': 'GOMEZ', 'Prenom': 'Diego'}
{'IdSportif': 9, 'Nom': 'ANTUNES', 'Prenom': 'Gaêlle'}
{'IdSportif': 10, 'Nom': 'BLANDET', 'Prenom': 'Arnaud'}
{'IdSportif': 11, 'Nom': 'BONNET', 'Prenom': 'Christophe'}
{'IdSportif': 12, 'Nom': 'BORREL', 'Prenom': 'Benoît'}
{'IdSportif': 13, 'Nom': 'BOYON', 'Prenom': 'Julien'}
{'IdSportif': 14, 'Nom': 'CAILLIOT', 'Prenom': 'Stéphanie'}
{'IdSportif': 15, 'Nom': 'CHAMPENOIS', 'Prenom': 'Cédric'}
{'IdSportif': 16, 'Nom': 'COLAS', 'Prenom': 'Michaël'}
{'IdSportif': 17, 'Nom': 'COLOMB', 'Prenom': 'Michaël'}
{'IdSportif': 18, 'Nom': 'HOSNI', 'Prenom': 'Leila'}
{'IdSportif': 19, 'Nom': 'TESTEMONT', 'Prenom': 'Laurent'}
{'IdSportif': 20, 'Nom': 'TIENER', 'Prenom': 'Thomas'}
{'IdSportif': 21, 'Nom': 'LE BOUCHER', 'Prenom': 'Denis'}
{'IdSportif': 22, 'Nom': 'LEDUFAUD', 'Prenom': 'Pierre'}
{'IdSportif': 23, 'Nom': 'LEGRAND', 'Prenom': 'David

Quels sont les gymnases de “Villetaneuse” ou de “Sarcelles” qui ont une surface de plus de
400 m2 ?

In [ ]:
for i in db.Gymnases.find({"$or":[{"$and":[{"Ville":"SARCELLES"},{"Surface":{"$gte":400}}]},{"$and":[{"Ville":"VILLETANEUSE"},{"Surface":{"$gte":400}}]}]},{"_id":0,"NomGymnase":1}):
    pprint(i)

In [48]:
for i in db.Gymnases.aggregate([{ "$group": {
        "_id": {
            "Ville": { "$toLower": "$Ville"} ,
        },
        
    }}
    ]):
    pprint(i)

{'_id': {'Ville': 'villetaneuse'}}
{'_id': {'Ville': 'saint denis'}}
{'_id': {'Ville': 'pierrefitte'}}
{'_id': {'Ville': 'garges'}}
{'_id': {'Ville': 'montmorency'}}
{'_id': {'Ville': 'stains'}}
{'_id': {'Ville': 'sarcelles'}}


In [42]:
for i in db.Gymnases.find_one():
    pprint(i)

'_id'
'IdGymnase'
'NomGymnase'
'Adresse'
'Ville'
'Surface'
'Seances'


In [53]:
db.Gymnases.aggregate([
   {
      "$project": {
         "Surface": {
            "$filter": {
               "input": "$Surface",
               "as" : "item",
               "cond": { "$gte": [ "$$item.Surface", 400 ] }
            }
         }
      }
   }
])

OperationFailure: input to $filter must be an array not int

Quels sont les sportifs (identifiant et nom) qui pratiquent du hand ball ?

In [ ]:
for i in db.Sportifs.find({"Sports.Jouer": "Hand ball"},{"_id": 1,"Nom": 1}):
    pprint(i)


Dans quels gymnases et quels jours y a t-il des séances de hand ball ?

In [137]:
for i in db.Gymnases.find(  {"Seances.Libelle": "Hand ball"},
                            {"_id": 0,"NomGymnase": 1,
                             "Ville": 1,"Seances.Jour": 1,
                             "Seances.Libelle": 1}):
    pprint(i)

{'NomGymnase': 'PAUL ELUARD',
 'Seances': [{'Jour': 'Samedi', 'Libelle': 'Basket ball'},
             {'Jour': 'Lundi', 'Libelle': 'Hand ball'},
             {'Jour': 'Lundi', 'Libelle': 'Hand ball'},
             {'Jour': 'Lundi', 'Libelle': 'Hand ball'},
             {'Jour': 'Lundi', 'Libelle': 'Hand ball'},
             {'Jour': 'lundi', 'Libelle': 'Hand ball'},
             {'Jour': 'Lundi', 'Libelle': 'Hand ball'},
             {'Jour': 'Dimanche', 'Libelle': 'Hand ball'},
             {'Jour': 'Dimanche', 'Libelle': 'Hand ball'},
             {'Jour': 'mardi', 'Libelle': 'Hand ball'},
             {'Jour': 'mercredi', 'Libelle': 'Hand ball'},
             {'Jour': 'Samedi', 'Libelle': 'Hand ball'},
             {'Jour': 'Samedi', 'Libelle': 'Hand ball'},
             {'Jour': 'Samedi', 'Libelle': 'Hand ball'},
             {'Jour': 'jeudi', 'Libelle': 'Hand ball'},
             {'Jour': 'lundi', 'Libelle': 'Hand ball'},
             {'Jour': 'lundi', 'Libelle': 'Hand ball'},
   

In [ ]:
for i in db.Gymnases.aggregate([{ "$unwind" : "$Seances" },
    { "$match": { "Seances.Libelle": "Hand ball" } },
    { "$project": {"_id": 0,"NomGymnase": 1,"Ville": 1,"Seances.Jour": 1,"Seances.Libelle": 1 }}]):
    pprint(i)

Dans quels gymnases peut-on jouer au hockey le mercredi apres 15H ?

In [ ]:
for i in db.Gymnases.aggregate([{ "$unwind" : "$Seances" },
    { "$match": {"Seances.Horaire":{"$gte":15}}},
    { "$match": {"Seances.Jour":"mercredi"}},
    { "$match": {"Seances.Libelle":"Hockey"}},
    { "$project": {"_id": 0,"NomGymnase": 1,"Seances.Jour":1,"Seances.Horaire": 1}}]):
    pprint(i)

Quels sportifs (identifiant et nom) ne pratiquent aucun sport ?

In [57]:
for i in db.Sportifs.aggregate([{ "$match" : {"Sports": { "$exists": False }}},
                                {}
                                { "$project": {"_id":0,"Nom":1}}]):
    pprint(i)

{'Nom': 'DROULLON'}
{'Nom': 'BELUAU'}
{'Nom': 'LENEVEU'}
{'Nom': 'DABON'}
{'Nom': 'CLERICE'}
{'Nom': 'COMES'}
{'Nom': 'BELZ'}
{'Nom': 'FERREIRA'}
{'Nom': 'GALLOIS'}


In [ ]:
for i in db.Sportifs.find({"Sports.Jouer" : { "$exists" : False }}):
    pprint(i)

Quels gymnases n’ont pas de séances le dimanche ?

In [40]:
for i in db.Gymnases.find(
    { "Seances.Jour" : { "$nin" : [ "dimanche", "Dimanche" ]}},
    { "_id": 0,
        "NomGymnase": 1,
        "Ville": 1,}):
    pprint(i)

{'NomGymnase': 'LAMARTINE', 'Ville': 'PIERREFITTE'}
{'NomGymnase': 'BREL', 'Ville': 'PIERREFITTE'}
{'NomGymnase': 'SAMOURAI', 'Ville': 'SARCELLES'}
{'NomGymnase': 'PABLO NERUDA', 'Ville': 'PIERREFITTE'}
{'NomGymnase': 'ARAGON', 'Ville': 'SAINT DENIS'}
{'NomGymnase': 'BELFEGOR', 'Ville': 'SAINT DENIS'}
{'NomGymnase': 'PASCAL', 'Ville': 'VILLETANEUSE'}
{'NomGymnase': 'SAINT EXUPERY', 'Ville': 'PIERREFITTE'}
{'NomGymnase': 'PAUL ELUARD', 'Ville': 'SARCELLES'}
{'NomGymnase': 'BRASSENS', 'Ville': 'SARCELLES'}
{'NomGymnase': 'VERLAINE', 'Ville': 'STAINS'}
{'NomGymnase': 'JULES FERRY', 'Ville': 'PIERREFITTE'}
{'NomGymnase': 'CAMUS', 'Ville': 'SARCELLES'}
{'NomGymnase': 'LUMIERES', 'Ville': 'SARCELLES'}
{'NomGymnase': 'SIMON', 'Ville': 'STAINS'}
{'NomGymnase': 'BARBARA', 'Ville': 'SAINT DENIS'}
{'NomGymnase': 'MERMOZ', 'Ville': 'VILLETANEUSE'}


Quels gymnases ne proposent que des séances de basket ball ou de volley ball ?

In [ ]:
 for i in db.Gymnases.find(   
    {
        "$nor": [
            { "Seances.Libelle": { "$ne": "Basket ball" }},
            { "Seances.Libelle": { "$ne": "Volley ball" }}
        ]
    },
    {
        "_id": 0,
        "NomGymnase": 1,
        "Ville": 1,
        "Seances.Libelle": 1
    }
):
        pprint(i)

Quels sont les entraîneurs qui sont aussi joueurs ?

In [37]:
for i in db.Sportifs.aggregate([{ "$unwind":"$Sports"},
                                { "$match" : { "$Sports.Jouer": { "$exists": False }}},
                                { "$project": {"_id":1,"Nom":1}}]):
    pprint(i)

OperationFailure: unknown top level operator: $Sports.Jouer

In [47]:
for i in db.Sportifs.find(
    {
        "Sports.Jouer" : { "$exists" : True },
        "Sports.Entrainer" : { "$exists" : True }
    },
    {
        "_id": 0,
        "Nom":  1,
        #"Sports.Entrainer":1,
        #"Sports.Jouer":1
    }
):
    pprint(i)

{'Nom': 'BOUTAHAR'}
{'Nom': 'HUE'}
{'Nom': 'DORLEANS'}
{'Nom': 'RETALDI'}
{'Nom': 'GOMEZ'}
{'Nom': 'TANQUE'}
{'Nom': 'DJELOUDANE'}
{'Nom': 'VASSEMON'}
{'Nom': 'VALIN'}
{'Nom': 'HEDDI'}
{'Nom': 'KALOMBO'}
{'Nom': 'GUERRAOUI'}
{'Nom': 'BOISSEAU'}
{'Nom': 'GUIGUI'}
{'Nom': 'CLERICE'}
{'Nom': 'TIZEGHAT'}
{'Nom': 'KERVADEC'}
{'Nom': 'ROUSSEL'}
{'Nom': 'SCHINK'}
{'Nom': 'STEMPUT'}
{'Nom': 'VAN CAUTER'}
{'Nom': 'LAZARRE'}
{'Nom': 'BELZ'}
{'Nom': 'HENRY'}


Quels sont les sportifs qui sont des conseillers ?

Pour le sportif “Kervadec” quel est le nom de son conseiller ?

In [72]:
aa=db.Sportifs.find({"Nom":"KERVADEC"})
print(aa[0]["IdSportifConseiller"])

1


In [76]:
for i in db.Sportifs.find(
    {
        "IdSportif": db.Sportifs.find({ "Nom": "KERVADEC" })[0]["IdSportifConseiller"]
    },
    {
        "_id": 0,
        "Sports": 0
    }
):
    pprint(i)

{'Age': 30,
 'IdSportif': 1,
 'Nom': 'BOUTAHAR',
 'Prenom': 'Abderahim',
 'Sexe': 'm'}


Quels entraîneurs entraînent du hand ball et du basket ball ?

In [78]:
for i in db.Sportifs.find(
    {
        "Sports.Entrainer": "Hand ball", 
        "Sports.Entrainer": "Basket ball" 
    },
    {
        "_id": 0,
        "Nom": 1,
        "Sports.Entrainer": 1
    }
):
    pprint(i)

{'Nom': 'BOUTAHAR',
 'Sports': {'Entrainer': ['Basket ball',
                          'Volley ball',
                          'Hand ball',
                          'Hockey',
                          'Badmington']}}
{'Nom': 'HUE',
 'Sports': {'Entrainer': ['Basket ball',
                          'Volley ball',
                          'Hand ball',
                          'Badmington']}}
{'Nom': 'DORLEANS',
 'Sports': {'Entrainer': ['Basket ball', 'Ping pong', 'Boxe']}}
{'Nom': 'KERVADEC',
 'Sports': {'Entrainer': ['Basket ball',
                          'Volley ball',
                          'Hand ball',
                          'Tennis',
                          'Hockey',
                          'Badmington',
                          'Ping pong',
                          'Boxe']}}
{'Nom': 'BELZ', 'Sports': {'Entrainer': 'Basket ball'}}
{'Nom': 'HENRY', 'Sports': {'Entrainer': ['Basket ball', 'Hand ball']}}


Quelle est la moyenne d’âge des sportives qui pratiquent du basket ball ?

In [88]:
for i in db.Sportifs.aggregate([
    { "$match": { "Sports.Jouer": "Basket ball", "Sexe": { "$in": [ "f", "F" ]} }},
    { "$group": { "_id": "NULL", "Moyenne": { "$avg": "$Age" }}},
    { "$project": {"_id":0,"Moyenne":1}}
]):
    pprint(i)

{'Moyenne': 24.6}


Quels sportifs n’ont pas de conseillers ?

In [91]:
for i in db.Sportifs.find({"IdSportifConseiller" : { "$exists" : False }},{"_id":0,"Nom":1}):
    pprint(i)

{'Nom': 'BOUTAHAR'}
{'Nom': 'CHAVANT'}
{'Nom': 'CLERICE'}
{'Nom': 'CHESNIER'}
{'Nom': 'BONE'}
{'Nom': 'CHAUVIN'}
{'Nom': 'RIQUIER'}
{'Nom': 'BONHOMME'}


Pour chaque sportif donner le nombre de sports qu’il arbitre

In [134]:
varMatch={"$match": { "Sports.Arbitrer": { "$exists": True }}}
varUnwind="{$unwind: "$Sports.Arbitrer"}
varGroup={"$group": { "_id": "$Nom", "nb_arbitrage": { "$sum": 1 }}}
varProject={"$project": {"_id":1,"nb_arbitrage":1}}

for i in db.Sportifs.aggregate([varMatch,varUnwind,varGroup,varProject]):
    pprint(i)

SyntaxError: invalid syntax (<ipython-input-134-dab6164df4a6>, line 2)